# Running dataset with basic convolutional model

If you want to see some basics about the data and the images, refer to the other notebooks

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torch.optim as optim

In [2]:
%load_ext autoreload
%autoreload 2

import data_handling as data
import model

## Creating dataset

The code for this dataset is included in ```data_handling.py```. 

In [3]:
needle_directory = '/Users/carlosolivares/be224-fp/data/NeedleImages/'
labels_path = os.path.join(needle_directory, 'Labels.csv')

data_transformer = transforms.Compose([transforms.ToTensor()])

dataset = data.NeedleImageDataset(
    path2data=needle_directory,
    path2labels=labels_path,
    transform = data_transformer
)

In [4]:
len_dataset = len(dataset)
len_train = int(0.8 * len_dataset)
len_val = len_dataset - len_train

train_ds, val_ds = random_split(dataset, [len_train, len_val])

print("train datset length:", len(train_ds))
print("validation dataset length:", len(val_ds))

train_dl = DataLoader(train_ds, batch_size=10, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=10, shuffle=False)

train datset length: 504
validation dataset length: 127


In [5]:
## checking the length of the dataloaders
print(len(train_dl))

51


In [6]:
"""
checking to see that the batches come appropriately shaped for input to model. 
The model accepts data in (batch, channels, height, width)
Our images are grayscale, so we stack the values to get 3 channels like an RGB
"""

for xb, yb in val_dl:
    print(xb.shape)
    print(yb.shape)
    break

torch.Size([10, 3, 512, 512])
torch.Size([10])


## Model option 1

This one has two output nodes with no output activation and ```torch.nn.CrossEntropyLoss``` as the objective function.

In [7]:
naivenet = model.NaiveNet()

In [9]:
out.shape #recall this is the multi-node output thing

torch.Size([10, 2])

### Training the model for ten epoch to see if it actually learns



In [10]:
weights = [.3027, .6973]
criterion = torch.nn.CrossEntropyLoss(reduction='sum', weight=torch.tensor(weights))
optimizer = optim.Adam(naivenet.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1, verbose=True)

Adjusting learning rate of group 0 to 1.0000e-02.


In [11]:
for epoch in range(4):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        naivenet.train()
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = naivenet(inputs)
        loss = criterion(outputs, labels.to(torch.int64))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0
    val_loss = model.get_validation_loss(naivenet, criterion, val_dl)
    print('validation loss after epoch {}: {}'.format((i+1), val_loss))
    scheduler.step()

print('Finished Training')

[1,    10] loss: 23.558
[1,    20] loss: 3.048
[1,    30] loss: 3.231
[1,    40] loss: 3.052
[1,    50] loss: 2.894
validation loss after epoch 51: 0.22011610091201902
Adjusting learning rate of group 0 to 1.0000e-03.
[2,    10] loss: 2.395
[2,    20] loss: 2.328
[2,    30] loss: 2.555
[2,    40] loss: 2.308
[2,    50] loss: 2.636
validation loss after epoch 51: 0.2083043836233184
Adjusting learning rate of group 0 to 1.0000e-04.
[3,    10] loss: 2.275
[3,    20] loss: 2.395
[3,    30] loss: 2.648
[3,    40] loss: 2.579
[3,    50] loss: 2.064
validation loss after epoch 51: 0.2011332568221205
Adjusting learning rate of group 0 to 1.0000e-05.
[4,    10] loss: 2.429
[4,    20] loss: 2.379
[4,    30] loss: 2.329
[4,    40] loss: 2.482
[4,    50] loss: 2.180
validation loss after epoch 51: 0.1975304376421951
Adjusting learning rate of group 0 to 1.0000e-06.
Finished Training


In [12]:
## Ok, it seems to be working, so now all we need is to figure out exactly how to get metrics and loss or whatever

In [13]:
## investigating how the crossentrupy loss works. From the docs: 

# Example of target with class indices
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
#output.backward()
# Example of target with class probabilities
#input = torch.randn(3, 5, requires_grad=True)
#target = torch.randn(3, 5).softmax(dim=1)
#output = loss(input, target)
#output.backward()

### Looking at how the loss function works

In [14]:
input

tensor([[-0.6759, -0.8502,  0.7985, -0.5464,  1.2984],
        [-1.1789, -1.6076, -0.6740,  1.3447, -0.6631],
        [-1.3812,  0.5992,  2.3664,  0.8805,  0.3675]], requires_grad=True)

In [15]:
target

tensor([1, 0, 2])

In [16]:
output

tensor(2.0512, grad_fn=<NllLossBackward>)

In [17]:
outputs

tensor([[-0.4508,  0.3201],
        [-0.4884,  0.3111],
        [-0.4619,  0.3550],
        [-0.5006,  0.3499]], grad_fn=<AddmmBackward>)

In [18]:
labels

tensor([0, 0, 1, 1])

In [19]:
loss(outputs, labels)

tensor(0.7609, grad_fn=<NllLossBackward>)

In [20]:
len(val_dl.dataset)

127

In [ ]:
# the next thing to do is actually get the metrics from model and stuff. 
# looks like you might need an lr schedulaer

## Model option 2

Same model but with one output node, sigmoid activation, and ```torch.nn.BCELoss```. 

In [47]:
sigmoid_model = model.NaiveNetSigmoid()

In [48]:
for xb, yb in val_dl:
    out = sigmoid_model(xb)
    print(yb)
    break

tensor([1., 1., 0., 1., 0., 1., 1., 0., 1., 1.], dtype=torch.float64)


In [49]:
out

tensor([[0.4877],
        [0.4873],
        [0.4885],
        [0.4867],
        [0.4873],
        [0.4872],
        [0.4884],
        [0.4864],
        [0.4860],
        [0.4881]], grad_fn=<SigmoidBackward>)

In [54]:
criterion = torch.nn.BCELoss(reduction='mean')
optimizer = optim.Adam(naivenet.parameters())
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        sigmoid_model.train()
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = sigmoid_model(inputs)
        #print(outputs.dtype)
        #print(labels.view(10,1).dtype)
        loss = criterion(outputs, labels.view(-1, 1).to(torch.float32))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0
    val_loss = model.get_validation_loss(sigmoid_model, criterion, val_dl)
    print('validation loss after epoch {}: {}'.format((i+1), val_loss))

print('Finished Training')

[1,    10] loss: 0.702
[1,    20] loss: 0.705
[1,    30] loss: 0.705
[1,    40] loss: 0.703
[1,    50] loss: 0.705


ValueError: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.

In [15]:
sum([ data.change_to_numeric_binary(label) for label in dataset.example_label_map.values()])

440.0

In [16]:
len(dataset.example_label_map.values())

631

In [ ]:
weights = [.3027, .6973]